## Writing the process_image Client Node

The second node that you’ll write in this project is the `process_image` node. This client node will subscribe to the robot’s camera images and analyze them to determine the position of the white ball. Once the ball position is determined, the client node will request a service from the `drive_bot` server node to drive the robot toward the ball. The robot can drive either left, right, or forward, depending on the ball’s position inside the image.

After you write this node, place the white ball in front of the robot’s camera. If everything works, your node should analyze the image, detect the ball’s position, and then request a `ball_chaser/command_robot` service to drive the robot towards the ball!

---

### Reference

The `process_image.cpp` client node is similar to the `look_away.cpp` client node that you wrote earlier. Both nodes contain a ROS subscriber and client. Please review the `look_away.cpp` node before you start coding the `process_image.cpp` node.

---

### Analyzing the Images

To identify the ball’s presence and position inside the image, you will use a simple approach:

1. Search for white pixels inside the array image. Since the ball is the only object in the world that is white, white pixels indicate the ball’s presence.
2. Identify the ball’s position with respect to the camera: left, middle, or right side of the image.

![img](lesson4p15_img1.png)

---

### Steps to Write `process_image.cpp`

#### Code Hints

```cpp
#include "ros/ros.h"
#include "ball_chaser/DriveToTarget.h"
#include <sensor_msgs/Image.h>

// Define a global client that can request services
ros::ServiceClient client;

// This function calls the command_robot service to drive the robot in the specified direction
void drive_robot(float lin_x, float ang_z)
{
    ball_chaser::DriveToTarget srv;
    srv.request.linear_x = lin_x;
    srv.request.angular_z = ang_z;

    if (!client.call(srv)) {
        ROS_ERROR("Failed to call service command_robot");
    }
}

// This callback function continuously executes and reads the image data
void process_image_callback(const sensor_msgs::Image img)
{
    int white_pixel = 255;
    bool ball_found = false;
    int ball_position = -1;

    for (int i = 0; i < img.height * img.step; i += 3) {
        if (img.data[i] == white_pixel && img.data[i + 1] == white_pixel && img.data[i + 2] == white_pixel) {
            ball_position = (i % img.step) / 3;
            ball_found = true;
            break;
        }
    }

    if (!ball_found) {
        drive_robot(0.0, 0.0);
        return;
    }

    int image_third = img.width / 3;
    if (ball_position < image_third) {
        drive_robot(0.0, 0.5); // Turn left
    } else if (ball_position < 2 * image_third) {
        drive_robot(0.5, 0.0); // Move forward
    } else {
        drive_robot(0.0, -0.5); // Turn right
    }
}

int main(int argc, char** argv)
{
    ros::init(argc, argv, "process_image");
    ros::NodeHandle n;

    client = n.serviceClient<ball_chaser::DriveToTarget>("/ball_chaser/command_robot");

    ros::Subscriber sub1 = n.subscribe("/camera/rgb/image_raw", 10, process_image_callback);

    ros::spin();

    return 0;
}
```

---

### Edit `CMakeLists.txt`

In addition to the dependencies for `drive_bot.cpp`, add these for `process_image.cpp`:

```cmake
add_executable(process_image src/process_image.cpp)
target_link_libraries(process_image ${catkin_LIBRARIES})
add_dependencies(process_image ball_chaser_generate_messages_cpp)
```

---

### Build the Package

Compile your updated package:

```bash
$ cd ~/catkin_ws/
$ catkin_make
```

---

### Launch File

Edit the `ball_chaser.launch` file and add the `process_image` node:

```xml
<launch>
  <!-- The drive_bot node -->
  <node name="drive_bot" type="drive_bot" pkg="ball_chaser" output="screen"/>

  <!-- The process_image node -->
  <node name="process_image" type="process_image" pkg="ball_chaser" output="screen"/>
</launch>
```

---

### Test `process_image`

1. **Launch the Robot:**
   ```bash
   $ cd ~/catkin_ws/
   $ source devel/setup.bash
   $ roslaunch my_robot world.launch
   ```

2. **Run `ball_chaser` Launch File:**
   ```bash
   $ cd ~/catkin_ws/
   $ source devel/setup.bash
   $ roslaunch ball_chaser ball_chaser.launch
   ```

3. **Visualize the Camera Feed:**
   - Subscribe to the `/camera/rgb/image_raw` topic from RViz or run:

     ```bash
     $ rosrun rqt_image_view rqt_image_view
     ```

4. **Test the Robot:**
   - Place the white ball at different positions in front of the robot and observe its behavior. The robot should analyze the image, detect the ball’s position, and drive toward it!

![img](lesson4p15_img2.png)
